In [ ]:
! pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# ! cat ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d oliviervha/crypto-news --force
! kaggle datasets download -d stepantita/crypto-news-price-impact --force

 85% 2.00M/2.34M [00:01<00:00, 2.13MB/s]
100% 2.34M/2.34M [00:01<00:00, 2.11MB/s]
100% 456k/456k [00:00<00:00, 629kB/s]
100% 456k/456k [00:00<00:00, 628kB/s]


In [ ]:
! unzip crypto-news.zip

Archive:  crypto-news.zip
  inflating: cryptonews.csv          


In [ ]:
! unzip crypto-news-price-impact.zip

Archive:  crypto-news-price-impact.zip
replace BING_source_title_description_bin_clf.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BING_source_title_description_bin_clf.csv  
replace news-1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yA
  inflating: news-1.csv              
replace news-2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Ay
  inflating: news-2.csv              
  inflating: news-3.csv              
  inflating: news-5.csv              
  inflating: news-7.csv              
  inflating: news-8.csv              
  inflating: news-9.csv              
  inflating: source_title_description_bin_clf.csv  


In [ ]:
import math
import json
from collections import Counter
import random
import os

from io import StringIO
from html.parser import HTMLParser

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding

from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
SEED = 42

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=SEED)

In [ ]:
def on_gpu(f):
    def wrapper(*args):
        if torch.cuda.is_available():
            return f(*args)
        else:
            print('cuda unavailable')
    return wrapper

In [ ]:
if torch.cuda.is_available():
    ! pip install pynvml
    from pynvml import *
    from numba import cuda

@on_gpu
def print_gpu_utilization(dev_id):
    try:
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(dev_id)
        info = nvmlDeviceGetMemoryInfo(handle)
        print(f"GPU memory occupied: {info.used//1024**2} MB.")
    except Exception as e:
        print(e)

@on_gpu
def free_gpu_cache(dev_id=0):
    print("Initial GPU Usage")
    print_gpu_utilization(dev_id)

    torch.cuda.empty_cache()

    print("GPU Usage after emptying the cache")
    print_gpu_utilization(dev_id)

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.4 MB/s eta 0:00:00


In [ ]:
device_id = 0

In [ ]:
device = torch.device(f'cuda:{device_id}' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(device)

In [ ]:
MODEL_NAME = 'bert-base-cased'
MODELS_PATH = 'models'
DATASET_NAME = 'crypto-news-plus' # hate_speech18, hatexplain
NUM_LABELS = 2

NUM_EPOCHS = 1
BATCH_SIZE = 32
MAX_SEQ_LEN = 512
LEARNING_RATE = 2e-5
MAX_GRAD_NORM = 1000

In [ ]:
data = pd.read_csv('source_title_description_bin_clf.csv')
data

,text,label
0,Source: CoinTelegraph Title: Bitcoin options d...,1
1,Source: The Daily Hodl Title: Crypto Exchange ...,0
2,Source: The Daily Hodl Title: BitKeep Wallet G...,0
3,Source: Bitcoin Magazine Title: Kazakhstan Pre...,0
4,Source: CoinTelegraph Title: The blockchain tr...,0
...,...,...
1313,Source: U.Today Title: Ethereum (ETH) Price An...,0
1314,Source: ZyCrypto Title: Digital Assets Amongst...,0
1315,"Source: The Daily Hodl Title: Cardano, Litecoi...",1
1316,Source: ZyCrypto Title: Binance plans to inves...,1


In [ ]:
data_unsupervised = pd.read_csv('BING_source_title_description_bin_clf.csv')
data_unsupervised

,text,label
0,Source: CoinTelegraph Title: Allbridge to firs...,0
1,Source: CryptoPotato Title: Bitcoin Hodl Patte...,1
2,Source: CoinTelegraph Title: ETH hits 7-month ...,1
3,Source: CoinTelegraph Title: Marathon Digital ...,1
4,Source: CryptoPotato Title: Brazilian Finance ...,0
...,...,...
616,Source: CryptoPotato Title: El Salvador Launch...,0
617,Source: CoinTelegraph Title: Worlds collide at...,1
618,Source: CoinTelegraph Title: Euler Finance hac...,1
619,Source: CryptoNews Title: Bitcoin Price and Et...,1


In [ ]:
# data = pd.concat([data, data_unsupervised], axis=0).reset_index(drop=True)

,text,label
0,Source: CoinTelegraph Title: Bitcoin options d...,1
1,Source: The Daily Hodl Title: Crypto Exchange ...,0
2,Source: The Daily Hodl Title: BitKeep Wallet G...,0
3,Source: Bitcoin Magazine Title: Kazakhstan Pre...,0
4,Source: CoinTelegraph Title: The blockchain tr...,0
...,...,...
1934,Source: CryptoPotato Title: El Salvador Launch...,0
1935,Source: CoinTelegraph Title: Worlds collide at...,1
1936,Source: CoinTelegraph Title: Euler Finance hac...,1
1937,Source: CryptoNews Title: Bitcoin Price and Et...,1


In [ ]:
dataset = DatasetDict({
    'train': Dataset.from_list(data.to_dict('records'))
})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1318
    })
})

In [ ]:
train_split, test_split = train_test_split(dataset['train'], test_size=0.5, random_state=SEED)

In [ ]:
dataset['test'] = Dataset.from_dict(test_split)

test_split, val_split = train_test_split(dataset['test'], test_size=0.2, random_state=SEED)

['Source: cryptopotato.com Title: Bitcoin Rises Briefly as Fed Chair Promises More Rate Hikes Description: The Federal Reserve may have to keep rate hikes going for longer than previously expected.',
 'Source: Live Bitcoin News Title: Australia Hints Crypto Regulation Will Arrive in 2023 Description: The government of Australia has hinted it’s going to be implementing cryptocurrency regulations in 2023. The announcement comes on the heels of the FTX collapse, which will likely go down as one of the biggest embarrassments of the crypto space. Australia Could Regulate Crypto Next Year Australian Treasurer Jim Chalmers has stated that regulators are...',
 'Source: Finbold Title: New data shows strong signals of a ‘hopeful future’ for Bitcoin Description: Although 2022 has been a challenging year for the cryptocurrency market, Bitcoin (BTC) is recording positive results in terms of … Continued',
 'Source: NewsBTC Title: Toncoin (TON) Shoots Up 50% In 7 Days – This Telegram Feature Cranks U

In [ ]:
dataset['train'] = Dataset.from_dict({
    'text': train_split['text'] + data_unsupervised['text'].tolist(),
    'label': train_split['label'] + data_unsupervised['label'].tolist(),
})
dataset['test'] = Dataset.from_dict(test_split)
dataset['val'] = Dataset.from_dict(val_split)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1280
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 527
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 132
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
tokenized_dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=MAX_SEQ_LEN, return_tensors='pt'), batched=True)
tokenized_dataset.set_format('torch', device=device)
tokenized_dataset

Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

Map:   0%|          | 0/527 [00:00<?, ? examples/s]

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1280
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 527
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 132
    })
})

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS).to(device)
base_model

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
for p in base_model.bert.parameters():
    p.requires_grad = False
# for p in base_model.bert.encoder.layer[-1].parameters():
#     p.requires_grad = True

In [ ]:
count_parameters(base_model)

1538

In [ ]:
def eval(f):
    def wrapper(model, *args, **kwargs):
        model.eval()
        return f(model, *args, **kwargs)
    return wrapper

def train(f):
    def wrapper(model, *args, **kwargs):
        model.train()
        return f(model, *args, **kwargs)
    return wrapper

In [ ]:
@train
def train_epoch(model, train_dataloader, optimizer):
    train_loss = 0.0
    train_preds = []
    train_labels = []

    for step, batch in enumerate(tqdm(train_dataloader, total=len(train_dataloader))):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        targets = batch['label'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets) # (B, Seq_Len, 2)

        loss, logits = outputs.loss, outputs.logits

        probs = F.softmax(logits, dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        train_preds += pred.detach().tolist()
        train_labels += [l.item() for l in targets.cpu()]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    return train_loss, train_preds, train_labels

@eval
def eval_epoch(model, val_dataloader):
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():

        for step, batch in enumerate(tqdm(val_dataloader, total=len(val_dataloader))):
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            targets = batch['label'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)

            loss, logits = outputs.loss, outputs.logits

            probs = F.softmax(logits, dim=-1).cpu()
            pred = torch.argmax(probs, dim=-1) # (B)
            val_preds += pred.detach().tolist()
            val_labels += [l.item() for l in targets.cpu()]

            val_loss += loss.item()
    return val_loss, val_preds, val_labels

In [ ]:
def training(model, train_data, val_data, config, balance='weighted'):
    model = model.to(device)

    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=config['lr'],
        weight_decay=config['weight_decay']
    )

    num_train_steps = int(len(train_data) / config['batch_size'] * config['epochs'])

    print(f'Train steps: {num_train_steps}')

    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=2 * config['batch_size'])

    history = {
        'train_losses': [],
        'val_losses': [],
        'train_acc': [],
        'val_acc': [],
        'train_f1': [],
        'val_f1': [],
        'train_precision': [],
        'val_precision': [],
        'train_recall': [],
        'val_recall': [],
        'train_roc_auc': [],
        'val_roc_auc': [],
        'train_precision_recall': [],
        'val_precision_recall': [],
    }

    for epoch_num in range(config['epochs']):
        print(f'Epoch: {epoch_num + 1}')

        # train stage
        train_loss, train_preds, train_labels = train_epoch(model, train_dataloader, optimizer)

        # eval stage
        val_loss, val_preds, val_labels = eval_epoch(model, val_dataloader)

        # metrics
        train_acc = accuracy_score(train_labels, train_preds)
        val_acc = accuracy_score(val_labels, val_preds)
        train_f1 = f1_score(train_labels, train_preds, average=balance)
        val_f1 = f1_score(val_labels, val_preds, average=balance)
        train_precision = precision_score(train_labels, train_preds, average=balance)
        val_precision = precision_score(val_labels, val_preds, average=balance)
        train_recall = recall_score(train_labels, train_preds, average=balance)
        val_recall = recall_score(val_labels, val_preds, average=balance)
        train_roc_auc = roc_auc_score(train_labels, train_preds, average=balance)
        val_roc_auc = roc_auc_score(val_labels, val_preds, average=balance)
        train_precision_recall = average_precision_score(train_labels, train_preds, average=balance)
        val_precision_recall = average_precision_score(val_labels, val_preds, average=balance)

        history['train_losses'].append(train_loss / len(train_dataloader))
        history['val_losses'].append(val_loss / len(val_dataloader))
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)
        history['train_precision'].append(train_precision)
        history['val_precision'].append(val_precision)
        history['train_recall'].append(train_recall)
        history['val_recall'].append(val_recall)
        history['train_roc_auc'].append(train_roc_auc)
        history['val_roc_auc'].append(val_roc_auc)
        history['train_precision_recall'].append(train_precision_recall)
        history['val_precision_recall'].append(val_precision_recall)

        print()
        print(f'Train loss: {train_loss / len(train_dataloader)} | Val loss: {val_loss / len(val_dataloader)}')
        print(f'Train acc: {train_acc} | Val acc: {val_acc}')
        print(f'Train f1: {train_f1} | Val f1: {val_f1}')
        print(f'Train precision: {train_precision} | Val precision: {val_precision}')
        print(f'Train recall: {train_recall} | Val recall: {val_recall}')
        print(f'Train AUCROC: {train_roc_auc} | Val AUCROC: {val_roc_auc}')
        print(f'Train AUPRC: {train_precision_recall} | Val AUPRC: {val_precision_recall}')

    free_gpu_cache(device_id)
    return history

In [ ]:
ids = tokenized_dataset['test'][0]['input_ids'].unsqueeze(0)
mask = tokenized_dataset['test'][0]['attention_mask'].unsqueeze(0)
targets = tokenized_dataset['test'][0]['label'].unsqueeze(0)

ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)

out = base_model(ids, mask, labels=targets)
out.loss

tensor(0.3002, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
config = {
    'epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'gradient_accumulation_steps': 1,
    'fp16': False,
    'lr': LEARNING_RATE,
    'max_grad_norm': MAX_GRAD_NORM,
    'weight_decay': 0.01,
}

In [ ]:
history = training(base_model, tokenized_dataset['train'], tokenized_dataset['val'], config)

Train steps: 40
Epoch: 1


100%|██████████| 3/3 [00:04<00:00,  1.47s/it]


Train loss: 0.6921256512403489 | Val loss: 0.6741787592569987
Train acc: 0.57265625 | Val acc: 0.6287878787878788
Train f1: 0.4679815738054288 | Val f1: 0.48548273431994354
Train precision: 0.5359461886710685 | Val precision: 0.39537419651056016
Train recall: 0.57265625 | Val recall: 0.6287878787878788
Train AUCROC: 0.5072240505158786 | Val AUCROC: 0.5
Train AUPRC: 0.4281847589832541 | Val AUPRC: 0.3712121212121212
Initial GPU Usage
GPU memory occupied: 3977 MB.
GPU Usage after emptying the cache
GPU memory occupied: 1767 MB.



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def plot_results(history, do_val=True):
    fig, ax = plt.subplots(figsize=(8, 8))

    x = list(range(0, len(history['train_losses'])))

    # loss

    ax.plot(x, history['train_losses'], label='train_loss')

    if do_val:
        ax.plot(x, history['val_losses'], label='val_loss')

    plt.title('Train / Validation Loss')
    plt.legend(loc='upper right')

    # accuracy

    if 'train_acc' in history:
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.plot(x, history['train_acc'], label='train_acc')

        if do_val:
            ax.plot(x, history['val_acc'], label='val_acc')

    plt.title('Train / Validation Accuracy')
    plt.legend(loc='upper right')

    # f1-score

    if 'train_f1' in history:
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.plot(x, history['train_f1'], label='train_f1')

        if do_val:
            ax.plot(x, history['val_f1'], label='val_f1')

        plt.title('Train / Validation F1')
        plt.legend(loc='upper right')

    # precision

    if 'train_precision' in history:
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.plot(x, history['train_precision'], label='train_precision')

        if do_val:
            ax.plot(x, history['val_precision'], label='val_precision')

        plt.title('Train / Validation Precision')
        plt.legend(loc='upper right')

    # recall

    if 'train_recall' in history:
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.plot(x, history['train_recall'], label='train_recall')

        if do_val:
            ax.plot(x, history['val_recall'], label='val_recall')

        plt.title('Train / Validation Recall')
        plt.legend(loc='upper right')

    fig.show()

In [ ]:
# plot_results(history)

In [ ]:
# torch.save(base_model.state_dict(), f'{MODELS_PATH}/{DATASET_NAME}_{MODEL_NAME.replace("/", "_")}_{NUM_EPOCHS}.bin')

In [ ]:
val_dataset = tokenized_dataset['test']

In [ ]:
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2 * config['batch_size'])

In [ ]:
val_loss, val_preds, val_labels = eval_epoch(base_model, val_dataloader)

100%|██████████| 9/9 [00:16<00:00,  1.87s/it]


In [ ]:
val_acc = accuracy_score(val_labels, val_preds)
val_f1 = f1_score(val_labels, val_preds, average='weighted')
val_precision = precision_score(val_labels, val_preds, average='weighted')
val_recall = recall_score(val_labels, val_preds, average='weighted')
val_roc_auc = roc_auc_score(val_labels, val_preds, average='weighted')
val_precision_recall = average_precision_score(val_labels, val_preds, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f'Val loss: {val_loss / len(val_dataloader)}')
print(f'Val acc: {val_acc}')
print(f'Val f1: {val_f1}')
print(f'Val precision: {val_precision}')
print(f'Val recall: {val_recall}')
print(f'Val ROCAUC: {val_roc_auc}')
print(f'Val AUPRC: {val_precision_recall}')

Val loss: 0.6459793845812479
Val acc: 0.6622390891840607
Val f1: 0.527674525400085
Val precision: 0.43856061124333434
Val recall: 0.6622390891840607
Val ROCAUC: 0.5
Val AUPRC: 0.3377609108159393
